In [12]:
import pandas as pd
import numpy as np
from copy import deepcopy
from pdb import set_trace
from sklearn.tree import DecisionTreeClassifier

In [1]:
class my_AdaBoost:

    def __init__(self, base_estimator = None, n_estimators = 50):
        # base_estimator: the base classifier class, e.g. my_DT
        # n_estimators: # of base_estimator rounds
        self.base_estimator = base_estimator
        self.n_estimators = int(n_estimators)
        self.estimators = [deepcopy(self.base_estimator) for i in range(self.n_estimators)]

    def fit(self, X, y):
        # X: pd.DataFrame, independent variables, float
        # y: list, np.array or pd.Series, dependent variables, int or str
        self.classes_ = list(set(list(y)))
        #print(self.classes_)
        
        k = len(self.classes_)
        #print(k)
        
        n = len(y)
        #print(n)
        
        w = np.array([1.0 / n] * n)
        #print(w)
        #print(len(w))
        
        labels = np.array(y)
        #print(labels)
        
        self.alpha = []
        
        for i in range(self.n_estimators):
            # Sample with replacement from X, with probability w
            sample = np.random.choice(n, n, p=w)
            #print(sample)
            
            # Train base classifier with sampled training data
            sampled = X.iloc[sample]
            #print(sampled)
            
            sampled.index = range(len(sample))
            #print(sampled.index)
            #print(labels[sample])
            
            self.estimators[i].fit(sampled, labels[sample])
            
            predictions = self.estimators[i].predict(X)
            #print(predictions)
            
            diffs = np.array(predictions) != y
            #print(len(diffs))
            
            # Compute error rate and alpha for estimator i
            error = np.sum(diffs * w)
            #print(error)
            #print(1 - 1.0 / k)
            
            while error >= (1 - 1.0 / k):
                w = np.array([1.0 / n] * n)
                
                sample = np.random.choice(n, n, p=w)
                
                # Train base classifier with sampled training data
                sampled = X.iloc[sample]
                
                sampled.index = range(len(sample))
                
                self.estimators[i].fit(sampled, labels[sample])
                
                predictions = self.estimators[i].predict(X)
                #print(predictions)
                
                diffs = np.array(predictions) != y
                # Compute error rate and alpha for estimator i
                
                error = np.sum(diffs * w)
            
            # Compute alpha for estimator i
            EPS = 1e-10
            self.alpha.append(np.log(k - 1) + (np.log((1 - error) / (error + EPS))))
            #print(-self.alpha[-1])

            Update wi
            for i in range(n):
                if(diffs[i]):
                    w[i] *= np.exp(self.alpha[-1])
                else:
                    w[i] = w
                    
            for i in range(n):
                if(diffs[i]):
                    w[i] = w
                else:
                    w[i] *= np.exp(self.alpha[-1])
                   
                    
                    
                
            
            #normalisation
            w = w / np.sum(w)
            #print(np.sum(w))
            #print(w)

        # Normalize alpha
        self.alpha = self.alpha / np.sum(self.alpha)
#         print(self.alpha)
#         print(w)
#         print(error)
#         print(predictions)
#         print(self.estimators)
        return

    def predict(self, X):
        # X: pd.DataFrame, independent variables, float
        # return predictions: list
        probs = self.predict_proba(X)
        predictions = [self.classes_[np.argmax(prob)] for prob in probs.to_numpy()]
        #print(predictions)
        return predictions

    def predict_proba(self, X):
        # X: pd.DataFrame, independent variables, float
        # prob: what percentage of the base estimators predict input as class C
        # prob(x)[C] = sum(alpha[j] * (base_model[j].predict(x) == C))
        # return probs = pd.DataFrame(list of prob, columns = self.classes_)
        # write your code below
        
        probs = {}
        
        p = 1
        
        X_cols_vals = X[X.columns]
        #print(X_cols_vals)
        
        
        for label in self.classes_:
            for i in range(self.n_estimators):
                probs[label] = (self.alpha[i] * ((self.estimators[i].predict(X)) == label))
        
        for labels_i,values_i in probs.items():
            probs[labels_i] = np.argmax(np.sum(probs[labels_i][values_i]))
                
                    
#         for label in self.classes_:
#             for i in range(self.n_estimators): #true =1 false = 0
#                 result = ((self.estimators[i].predict(X)) == label).astype(int)
#                 result[result == 0] = -1
#                 #print((result))
#                 for j in range(len(result)):
#                     probs[label] = (self.alpha[i] * result)
                        
                #probs[label] = (self.alpha[i] * ((self.estimators[i].predict(X)) == label))
                #probs[label] = (self.alpha[i] * (result))
            
        probs = pd.DataFrame(probs, columns=self.classes_)
        #probs = pd.Series(probs).to_frame(self.classes_)
        return probs

SyntaxError: invalid syntax (<ipython-input-1-86c453dd26bb>, line 82)

In [2]:
#  Load training data
data_train = pd.read_csv("../data/Iris_train.csv")
# Separate independent variables and dependent variables
independent = ["SepalLengthCm",	"SepalWidthCm",	"PetalLengthCm",	"PetalWidthCm"]
X = data_train[independent]
y = data_train["Species"]
# Train model
base_estimator = DecisionTreeClassifier(criterion = "entropy", max_depth = 1)
clf = my_AdaBoost(base_estimator=base_estimator, n_estimators = 50)
clf.fit(X, y)
# Load testing data
data_test = pd.read_csv("../data/Iris_test.csv")
X_test = data_test[independent]
# Predict
predictions = clf.predict(X_test)
#print(predictions)
# Predict probabilities
probs = clf.predict_proba(X_test)
# Print results
for i, pred in enumerate(predictions):
    print("%s\t%f" % (pred, probs[pred][i]))

NameError: name 'pd' is not defined